File built on the mechanics file

In [122]:
import requests
from lxml import etree
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sqlite3
import statsmodels.formula.api as smf

In [123]:
def add_dataframe_rank(dataframe, rank_column, is_ascending=False):
    """
    Adds a ranking column to the dataframe based on sort value
    
    Note that this function changes a dataframe "in place", so it does not return anything
    
    Attributes
    ----------
    dataframe: pandas DataFrame
        Dataframe that will have ranking column added to it
    
    rank_column: string
        Name of column by which dataframe will be sorted
        
    is_ascending: bool
        If true, sort dataframe in ascending order (smallest value first)
    """
    
    dataframe.sort_values(rank_column, ascending= is_ascending, inplace=True)
    dataframe.reset_index(inplace=True)
    dataframe['Rank'] = dataframe.index + 1 # so the top ranking game is #1 rather than #0
    dataframe.drop(['index'], axis=1, inplace=True)

In [124]:
con = sqlite3.connect('bgg.db')
cur = con.cursor()

In [125]:
query = '''
 SELECT t1.id, title, CAST(avg_rating AS FLOAT64) avg_rating, CAST(avg_bayes_rating AS FLOAT64),
        CAST(std as FLOAT64) std, category
 
 FROM TOP_GAMES_FULL t1
   LEFT JOIN BGG_CATEGORIES t2
     ON t1.id = t2.id
     
ORDER BY avg_rating DESC
'''

results = cur.execute(query)
result_li = [elem for elem in results.fetchall()]

In [126]:
df = pd.DataFrame(result_li, columns = ['id', 'Title','Score', 'BayesScore', 'StdDev', 'Category'])

In [127]:
pivot = pd.pivot_table(data=df, index='Category', values = ['id'], aggfunc='count')
pivot.columns = ['Games']
pivot.sort_values('Games', ascending=False,inplace=True)
pivot.head(10)

,Games
Category,
Card Game,246
Fantasy,210
Economic,200
Fighting,156
Science Fiction,130
Adventure,109
Wargame,102
Miniatures,97
Exploration,94


In [128]:
pivot.tail(5)

,Games
Category,
Electronic,3
Trivia,2
American Revolutionary War,2
Vietnam War,1
Expansion for Base-game,1


In [129]:
unpopular_cat_li = list(pivot[pivot['Games'] < 30].index)

In [130]:
# What are the top economic games?
df[df.Category == 'Economic'].head(10)

,id,Title,Score,BayesScore,StdDev,Category
30,284378,kanban_ev,8.70048,7.24979,1.20878,Economic
33,233078,twilight_imperium_fourth_edition,8.68469,8.25446,1.59933,Economic
40,224517,brass_birmingham,8.66929,8.40600,1.24659,Economic
61,220308,gaia_project,8.48599,8.17700,1.38992,Economic
79,167791,terraforming_mars,8.42470,8.27852,1.38438,Economic
87,182028,through_ages_new_story_civilization,8.40041,8.16310,1.50896,Economic
95,256960,pax_pamir_second_edition,8.38845,7.68404,1.41854,Economic
105,256916,concordia_venus,8.36289,7.29527,1.20657,Economic
134,193738,great_western_trail,8.30257,8.12938,1.26217,Economic
142,184267,mars,8.28442,7.69571,1.48998,Economic


In [131]:
# What is a fighting game? I don't think of any board game as being a fighting game
df[df.Category == 'Fighting'].head(10)

,id,Title,Score,BayesScore,StdDev,Category
3,169427,middara_unintentional_malum_act_1,8.86084,6.75708,1.58086,Fighting
8,235802,too_many_bones_undertow,8.76914,7.14766,1.29365,Fighting
12,174430,gloomhaven,8.76798,8.53089,1.63005,Fighting
17,291457,gloomhaven_jaws_lion,8.75730,8.25790,1.40857,Fighting
56,115746,war_ring_second_edition,8.50514,8.12414,1.48351,Fighting
74,187645,star_wars_rebellion,8.42688,8.17237,1.36940,Fighting
109,162886,spirit_island,8.35908,8.12427,1.45727,Fighting
118,294484,unmatched_cobble_fog,8.34178,6.89924,1.18596,Fighting
161,169786,scythe,8.22706,8.07352,1.43696,Fighting
196,274638,unmatched_robin_hood_vs_bigfoot,8.19061,6.68177,1.13646,Fighting


Fighting appears to mean "has combat." Different than the video game meaning of "fighting games," which usually entails a two-player game with combos etc

In [132]:
# My favorite: abstract strategy?
df[df.Category == 'Abstract Strategy'].head(10)

,id,Title,Score,BayesScore,StdDev,Category
489,249259,war_chest,7.95658,7.25427,1.33666,Abstract Strategy
513,319966,king_dead_second_edition,7.92003,6.70313,1.12215,Abstract Strategy
580,283155,calico,7.87500,7.29164,1.17497,Abstract Strategy
712,230802,azul,7.81294,7.69596,1.14777,Abstract Strategy
757,287954,azul_summer_pavilion,7.79682,7.39499,1.14047,Abstract Strategy
1025,7854,yinsh,7.69668,7.29183,1.32287,Abstract Strategy
1077,264241,mandala,7.67636,6.89158,1.08774,Abstract Strategy
1101,31999,tzaar,7.66416,6.94507,1.31992,Abstract Strategy
1138,163412,patchwork,7.64731,7.54058,1.14920,Abstract Strategy
1150,188,go,7.63898,7.32451,1.91084,Abstract Strategy


In [133]:
# Remove unpopular categories
df = df[~df['Category'].isin(unpopular_cat_li)]

In [134]:
cat_popularity = pd.pivot_table(df, index='Category',values='Score', aggfunc =['count','mean'])

In [135]:
cat_popularity.sort_values(('mean', 'Score'), ascending=False, inplace=True)
cat_popularity.head(10)

,count,mean
,Score,Score
Category,,
Wargame,102,7.744096
Space Exploration,35,7.736283
Miniatures,97,7.727345
Industry / Manufacturing,48,7.716122
Political,51,7.699332
Science Fiction,130,7.614947
Economic,200,7.613747
Adventure,109,7.610351


In [136]:
std_pivot = pd.pivot_table(df, index='Category', values =['Score'], aggfunc=['mean', 'std', 'count'])
std_pivot.sort_values(('mean', 'Score'), ascending=False).head(5)

,mean,std,count
,Score,Score,Score
Category,,,
Wargame,7.744096,0.359506,102
Space Exploration,7.736283,0.406687,35
Miniatures,7.727345,0.437759,97
Industry / Manufacturing,7.716122,0.421021,48
Political,7.699332,0.393972,51


Given the lack of variability for scores, let's test to see if there is a difference between the first and last score

In [137]:
from scipy.stats import ttest_ind

In [138]:
best_mechanic = df.loc[df['Category'] == 'Wargame', 'Score']

In [139]:
worst_mechanic = df.loc[df['Category'] == 'Party Game', 'Score']

In [140]:
ttest_ind(a=best_mechanic, b=worst_mechanic,equal_var=False)

Ttest_indResult(statistic=8.590847584285903, pvalue=1.3198776705344143e-13)

Pretty clearly significant. Side note: why is "wargame" one word and "party game" two words? Some questions can't be answered by data

In [141]:
# What are the most divisive categories?
std_pivot.sort_values(by=('std','Score'), ascending=False).head(5)

,mean,std,count
,Score,Score,Score
Category,,,
Adventure,7.610351,0.490190,109
Exploration,7.592264,0.460548,94
Fantasy,7.559552,0.443859,210
Miniatures,7.727345,0.437759,97
Territory Building,7.489967,0.432919,67


In [142]:
# The least?
std_pivot.sort_values(by=('std','Score'), ascending=True).head(5)

,mean,std,count
,Score,Score,Score
Category,,,
Party Game,7.266721,0.276530,42
Deduction,7.354518,0.294167,71
Renaissance,7.524183,0.325127,40
Abstract Strategy,7.312127,0.335520,43
City Building,7.437048,0.345630,89


Interesting: Party games are the lowest rated, but least divisive. People agree that they're the worst type of game in the top 1000 I guess. Similar for abstract strategy and deduction.

Next question: is the difference in ratings driven by weight?

In [143]:
query = '''
WITH inner_table AS( 
SELECT t1.id, title, CAST(avg_rating AS FLOAT64) avg_rating,
        CAST(std as FLOAT64) std, category, CAST(weight AS FLOAT64) weight,
        COUNT(t1.id) OVER (PARTITION BY category) NumOfGames
 
 FROM TOP_GAMES_FULL t1
   LEFT JOIN BGG_CATEGORIES t2
     ON t1.id = t2.id)
     
SELECT id, title, avg_rating, std, category, weight, NumOfGames

FROM inner_table

WHERE 1=1
    AND NumOfGames >= 30
     
ORDER BY avg_rating DESC
'''

results = cur.execute(query)
result_li = [elem for elem in results.fetchall()]

In [144]:
weight_df = pd.DataFrame(data = result_li, 
                         columns = ['id', 'title', 'score', 'std', 'category', 'weight', 'NumOfGames'])

In [145]:
weight_pivot = pd.pivot_table(data=weight_df, index='category',
                             values = ['weight', 'score'], aggfunc='mean')

In [146]:
weight_pivot.sort_values('weight',ascending=False).head(5)

,score,weight
category,,
Industry / Manufacturing,7.716122,3.478371
Political,7.699332,3.383667
Wargame,7.744096,3.222576
Civilization,7.598081,3.187869
Space Exploration,7.736283,3.178514


In [147]:
weight_pivot.corr()

,score,weight
score,1.000000,0.818833
weight,0.818833,1.000000


In [148]:
weight_pivot['weight_sq']= weight_pivot['weight'] ** 2

In [149]:
result = smf.ols(data=weight_pivot, formula='score~weight+weight_sq').fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  score   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     31.60
Date:                Sat, 18 Sep 2021   Prob (F-statistic):           4.12e-08
Time:                        20:11:22   Log-Likelihood:                 41.720
No. Observations:                  33   AIC:                            -77.44
Df Residuals:                      30   BIC:                            -72.95
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.1094      0.271     26.235      0.000       6.556       7.663
weight         0.0547      0.214      0.255      0.800      -0.383       0.492
weight_sq      0.0355      0.042      0.844      0.405      -0.050       0.121
==============================================================================
Omnibus:                        1.141   Durbin-Watson:                   1.848
Prob(Omnibus):                  0.565   Jarque-Bera (JB):                1.022
Skew:                           0.237   Prob(JB):                        0.600
Kurtosis:                       2.280   Cond. No.                         235.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [150]:
del weight_pivot['weight_sq']

Regression does not support a nortion of diminishing returns to weight. higher = better

Each point of weight is worth 0.233 towards a score. I will put forth a simplistic adjustment to scores in order to "remove" the impact of weight on a game's score. From there, I will see if the same rankings remaining

In [151]:
dedup_df = weight_df.drop_duplicates(subset='id', keep='first')

In [152]:
dedup_df.head()

,id,title,score,std,category,weight,NumOfGames
0,169427,middara_unintentional_malum_act_1,8.86084,1.58086,Adventure,3.9495,109
5,235802,too_many_bones_undertow,8.76914,1.29365,Adventure,4.0000,109
9,174430,gloomhaven,8.76798,1.63005,Adventure,3.8673,109
14,291457,gloomhaven_jaws_lion,8.75730,1.40857,Adventure,3.5590,109
19,246900,eclipse_second_dawn_galaxy,8.73618,1.33073,Civilization,3.5260,59


In [153]:
result = smf.ols(data=dedup_df, formula='score~weight').fit()
params = result.params
beta_0, beta_1 = params['Intercept'], params['weight']
print(beta_1)

0.26803772979497964


In [154]:
weight_df['score_adj'] = weight_df['score'] - beta_1 * weight_df['weight']

Next, we will compare the rankings with the original score and with the weight adjustment

In [155]:
new_pivot = pd.pivot_table(data=weight_df, index = 'category', values='score_adj', aggfunc=['count','mean'])
new_pivot.reset_index(inplace=True)
new_pivot.columns = ['category','count', 'score']

In [156]:
add_dataframe_rank(new_pivot, 'score')

In [157]:
# Need to make some fixes to the old pivot table before comparing
weight_pivot.reset_index(inplace=True)
weight_pivot.columns = ['category', 'score', 'weight']
add_dataframe_rank(weight_pivot,'score')

In [158]:
# Simplify adjusted dataframe
new_pivot = new_pivot[['category', 'Rank']]
new_pivot.rename(columns = {'Rank': 'Adj_Rank'}, inplace=True)

# Simplify original dataframe
weight_pivot = weight_pivot[['category','Rank']]
weight_pivot.rename(columns = {'Rank': 'Original_Rank'}, inplace=True)

In [161]:
merge_df = new_pivot.merge(weight_pivot, how='inner', on='category')
merge_df['Delta'] = merge_df['Adj_Rank'] - merge_df['Original_Rank']
merge_df

,category,Adj_Rank,Original_Rank,Delta
0,Miniatures,1,3,-2
1,Party Game,2,33,-31
2,Puzzle,3,19,-16
3,Space Exploration,4,2,2
4,Wargame,5,1,4
5,Movies / TV / Radio theme,6,16,-10
6,Mythology,7,9,-2
7,Adventure,8,8,0
8,Science Fiction,9,6,3
9,Exploration,10,11,-1
